In [1]:
import pandas as pd
import datetime as dt
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")

week_ago = week_ago.strftime("%Y-%m-%d")

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"
filepath = datapath + f"export_{today}.csv"


Bad key text.latex.preview in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       

### Import epoch level passive + GPS data

In [2]:
datapath1 = datapath + f"export_tiki_{today}/"
filepath_1 = datapath1 + "epoch_part0001.csv"
filepath_2 = datapath1 + "epoch_part0002.csv"
filepath_3 = datapath1 + "epoch_part0003.csv"
filepath_4 = datapath1 + "epoch_part0004.csv"
filepath_5 = datapath1 + "epoch_part0005.csv"
filepath_6 = datapath1 + "epoch_part0006.csv"
filepath_7= datapath1 + "epoch_part0007.csv"
filepath_8= datapath1 + "epoch_part0008.csv"
filepath_9= datapath1 + "epoch_part0009.csv"
filepath_10= datapath1 + "epoch_part0010.csv"

In [3]:
df_1 = pd.read_csv(filepath_1, encoding= "latin-1")
df_2 = pd.read_csv(filepath_2, encoding= "latin-1")
df_3 = pd.read_csv(filepath_3, encoding= "latin-1")
df_4 = pd.read_csv(filepath_4, encoding= "latin-1")
df_5 = pd.read_csv(filepath_5, encoding= "latin-1")
df_6 = pd.read_csv(filepath_6, encoding= "latin-1")
df_7 = pd.read_csv(filepath_7, encoding= "latin-1")
df_8 = pd.read_csv(filepath_8, encoding= "latin-1")
df_9 = pd.read_csv(filepath_9, encoding= "latin-1")
df_10 = pd.read_csv(filepath_10, encoding= "latin-1")

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_complete = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])
df_complete["customer"] = df_complete.customer.str.split("@").str.get(0)
df_complete["customer"] = df_complete.customer.str[:4]

In [5]:
df_complete["startTimestamp"] = pd.to_datetime(df_complete["startTimestamp"],unit='ms')
df_complete["createdAt"] = pd.to_datetime(df_complete["createdAt"],unit='ms')

df_complete["startTimestamp_day"] = df_complete.startTimestamp.dt.strftime('%Y/%m/%d')
df_complete["createdAt_day"] = df_complete.startTimestamp.dt.strftime('%Y/%m/%d')

df_complete["startTimestamp_hour"] = df_complete.startTimestamp.dt.hour
df_complete["createdAt_hour"] = df_complete.startTimestamp.dt.hour

### Import daily aggregated passive data

In [6]:
df = pd.read_csv(filepath, encoding= "latin-1")
df.drop(["Quelle", "Created At"], axis=1, inplace=True)

df["Endkunde"] = df.Endkunde.str.split("@").str.get(0)
df["Endkunde"] = df["Endkunde"].str[:4]
df.rename(columns={"Endkunde":"customer","Tag":"day", "Typ":"type","Wert(Fließkommazahl)": "float", "Wert(Ganzzahl)": "int"}, inplace=True)

df["day"] = pd.to_datetime(df["day"], format="%d.%m.%Y")
df["int"].fillna(df["float"], inplace=True)

In [7]:
df_int= df[["customer", "day", "type", "int"]]
df_int = df_int.pivot(
    index=["customer", "day"],
    columns="type",
    values="int")
df_int.reset_index(level="day", inplace=True)

### Check location data

In [8]:
df_loc_complete = df_complete[df_complete.type.isin(["Latitude", "Longitude"])]
df_loc_complete = df_loc_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           "timezoneOffset"]]
df_loc_complete["startTimestamp"] = (pd.to_datetime(df_loc_complete["startTimestamp"],unit='ms'))

In [9]:
df_loc_complete = df_loc_complete.groupby("customer")[["startTimestamp"]].max().rename_axis(None, axis=1).reset_index()

In [10]:
df_loc_complete.rename(columns={"startTimestamp":"last_day_GPS"}, inplace=True)


### Import montoring data

In [11]:
df_active = pd.read_csv(datapath + "active_participants.csv")

In [12]:
df_active.rename(columns = {"Pseudonym ": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

In [13]:
df_active["customer"] = df_active["customer"].str[:4]
df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]
df_active["for_id"] = df_active.for_id.str.strip()

### Import connection data

In [14]:
df_con = pd.read_csv(datapath + f"export_connection_{today}.csv", encoding= "latin-1")

In [15]:
df_con["Customer"] = df_con.Customer.str.split("@").str.get(0)
df_con["Customer"] = df_con["Customer"].str[:4]
df_con.drop(["Data Source", "Timestamp"], axis=1, inplace=True)
df_con.rename(columns = {"Customer": "customer"}, inplace=True)

## Merge dataframes

In [16]:
df_merged = pd.merge(df_int, df_active, on="customer")

In [17]:
df_merged = pd.merge(df_merged, df_con, on="customer")

In [18]:
grouped = df_merged.groupby(["customer"])["ema_id","for_id","start_ema","end_ema","study_version",
                                            "status","Is Connected","day"].max() # last day with passive data

grouped_active = grouped[~(grouped['status'] == "Abgeschlossen")]
grouped_active = grouped_active[~(grouped_active['status'].isin(["Dropout", "Pin_missing"]))]

grouped_active = grouped_active.rename_axis(None, axis=1).reset_index()
grouped_active.rename(columns = {"day": "last_day_passive"}, inplace=True)

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [19]:
df_merged = pd.merge(grouped_active, df_loc_complete, on="customer", how="outer")

In [25]:
df_merged.groupby("status")["customer"].nunique()

status
Erhebung_1_aktiv     7
Erhebung_2_aktiv     3
Post_Erhebung_1     89
Post_Erhebung_2      3
Name: customer, dtype: int64

## Check missing data

In [21]:
# no gps at all 

df_merged.loc[df_merged.last_day_GPS.isna()].for_id.tolist()

['FOR12022', 'FOR13010', 'FOR13019', 'FOR13013']

In [22]:
# no gps for > 7 days 

df_merged.loc[(df_merged.last_day_GPS < week_ago)& (df_merged.last_day_passive > week_ago)]["for_id"].tolist()

['FOR14038',
 'FOR14036',
 'FOR14014',
 'FOR12017',
 'FOR14065',
 'FOR11016',
 'FOR14016',
 'FOR11003',
 'FOR14031',
 'FOR12008',
 'FOR11040',
 'FOR12010',
 'FOR11034',
 'FOR13018']

In [23]:
# no gps and no pd for > 7 days

df_merged.loc[(df_merged.last_day_GPS < week_ago) & (df_merged.last_day_passive < week_ago)]["for_id"].tolist()

['FOR13002',
 'FOR13016',
 'FOR11022',
 'FOR12012',
 'FOR14064',
 'FOR14062',
 'FOR13023',
 'FOR14029',
 'FOR14013',
 'FOR11012']

In [24]:
# no pd since 7 days 
df_merged.loc[(df_merged.last_day_GPS > week_ago) & (df_merged.last_day_passive < week_ago)]["for_id"].tolist()

['FOR14055',
 'FOR14075',
 'FOR14009',
 'FOR14010',
 'FOR14049',
 'FOR14003',
 'FOR11001']